In [1]:
!pip install requests

In [2]:
client_id = '3365422ef607400b8a828303186993aa'
client_secret = 'fe290ecdb56a4fd28e8972f7e8e38c59'

In [3]:
import base64
import requests
import datetime
import pandas as pd
import numpy as np
from urllib.parse import urlencode

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = None
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_token_header(self):
        if self.client_id == None or self.client_secret == None:
            raise Exception('Invalid client ID/ client secret')
        authorization = base64.b64encode((f'{self.client_id}:{self.client_secret}').encode())
        header = {
            'Authorization': f'Basic {authorization.decode()}'
        }
        return header
    
    def get_token_data(self):
        request = {
            'grant_type': 'client_credentials'
        }
        return request
        
    def get_access_token(self):
        r = requests.post(self.token_url, data=self.get_token_data(), headers=self.get_token_header())
        if r.status_code not in range(200,299):
            return False
        data = r.json()
        expires_in = data['expires_in'] # seconds
        self.access_token_expires = datetime.datetime.now() + datetime.timedelta(seconds=expires_in)
        self.access_token = data['access_token']
        return True
    
    def get_resource_header(self):
        auth = self.get_access_token()
        if auth:
            headers = {
                "Authorization": f"Bearer {self.access_token}"
            }
            return headers
        return {}
    
    def get_resource(self, href):
        endpoint = href
        header = self.get_resource_header()
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        r = requests.get(endpoint, headers=header)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search(self, endpoint, query, search_type, limit = '20', offset = '0'):
        header = self.get_resource_header()
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        data = urlencode({
            'q': query,
            'type': search_type.lower(),
            'limit': limit,
            'offset': offset
        })
        lookup_url = f'{endpoint}?{data}'
        r = requests.get(lookup_url, headers = header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_resource_list(self, _type, ids):
        header = self.get_resource_header()
        endpoint = f'https://api.spotify.com/v1/{_type}'
        print('Token expires:', client.access_token_expires.strftime("%m/%d/%Y, %H:%M:%S"))
        data = urlencode({
            'ids': ids
        }).replace("%5B%27", "").replace("%27%2C+%27", ",").replace("%27%5D", "")
        lookup_url = f'{endpoint}?{data}'
        r = requests.get(lookup_url, headers = header)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [5]:
client = SpotifyAPI(client_id, client_secret)

In [6]:
# 1. Get FIFA Playlist tracks through search
data = client.search('https://api.spotify.com/v1/search', 'FIFA AND Soundtrack', 'playlist', '7')

Token expires: 07/20/2020, 21:52:38


In [7]:
playlist_endpoints = [] # Endpoint to list of tracks for each playlist
playlist_names = [] # Names of each playlist
playlist_num_tracks = [] # Number of tracks in each playlist
for item in data['playlists']['items']:
    playlist_endpoints.append(item['tracks']['href'])
    playlist_names.append(item['name'])
    playlist_num_tracks.append(item['tracks']['total'])

In [8]:
def fill_dict_song():
    album_name = []
    album_endpoint = []
    album_id = []

    artist1_name = []
    artist1_endpoint = []
    artist1_id = []
    
    artist2_name = []
    artist2_endpoint = []
    artist2_id = []
    
    artist3_name = []
    artist3_endpoint = []
    artist3_id = []

    track_name = []
    track_duration = []
    track_explicit = []
    track_popularity = []
    track_id = []

    for i, item in enumerate(tracks['items']):
        album_name.append(tracks['items'][i]['track']['album']['name'])
        album_endpoint.append(tracks['items'][i]['track']['album']['href'])
        album_id.append(tracks['items'][i]['track']['album']['id'])

        if len(tracks['items'][i]['track']['album']['artists']) == 1:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(None)
            artist2_endpoint.append(None)
            artist2_id.append(None)
            artist3_name.append(None)
            artist3_endpoint.append(None)
            artist3_id.append(None)
        elif len(tracks['items'][i]['track']['album']['artists']) == 2:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(tracks['items'][i]['track']['album']['artists'][1]['name'])
            artist2_endpoint.append(tracks['items'][i]['track']['album']['artists'][1]['href'])
            artist2_id.append(tracks['items'][i]['track']['album']['artists'][1]['id'])
            artist3_name.append(None)
            artist3_endpoint.append(None)
            artist3_id.append(None)
        elif len(tracks['items'][i]['track']['album']['artists']) == 3:
            artist1_name.append(tracks['items'][i]['track']['album']['artists'][0]['name'])
            artist1_endpoint.append(tracks['items'][i]['track']['album']['artists'][0]['href'])
            artist1_id.append(tracks['items'][i]['track']['album']['artists'][0]['id'])
            artist2_name.append(tracks['items'][i]['track']['album']['artists'][1]['name'])
            artist2_endpoint.append(tracks['items'][i]['track']['album']['artists'][1]['href'])
            artist2_id.append(tracks['items'][i]['track']['album']['artists'][1]['id'])
            artist3_name.append(tracks['items'][i]['track']['album']['artists'][2]['name'])
            artist3_endpoint.append(tracks['items'][i]['track']['album']['artists'][2]['href'])
            artist3_id.append(tracks['items'][i]['track']['album']['artists'][2]['id'])
        
        track_name.append(tracks['items'][i]['track']['name'])
        track_duration.append(tracks['items'][i]['track']['duration_ms'])
        track_explicit.append(tracks['items'][i]['track']['explicit'])
        track_popularity.append(tracks['items'][i]['track']['popularity'])
        track_id.append(tracks['items'][i]['track']['id'])
    
    dict_temp = {
        'track_name': track_name, 'track_duration': track_duration, 'track_explicit': track_explicit, 
        'track_popularity': track_popularity, 'track_id': track_id,
        'album_name': album_name, 'album_endpoint': album_endpoint, 'album_id': album_id, 
        'artist1_name': artist1_name, 'artist1_endpoint': artist1_endpoint, 'artist1_id': artist1_id,
        'artist2_name': artist2_name, 'artist2_endpoint': artist2_endpoint, 'artist2_id': artist2_id,
        'artist3_name': artist3_name, 'artist3_endpoint': artist3_endpoint, 'artist3_id': artist3_id
    }
    df_temp = pd.DataFrame(dict_temp)
    return df_temp

In [9]:
# 2. Get basic data on the tracks
dict_of_playlists = {}
for i, playlist_endpoint in enumerate(playlist_endpoints):
    tracks = client.get_resource(playlist_endpoint)
    dataframe = fill_dict_song()
    playlist_name = [playlist_names[i]]*len(dataframe)
    playlist_num_track = [playlist_num_tracks[i]]*len(dataframe)
    # Playlist Information
    dataframe.insert(0, 'num_tracks', playlist_num_track, True)
    dataframe.insert(0, 'playlist_name', playlist_name, True)
    # Add dataframe to dict
    dict_of_playlists[playlist_names[i]] = dataframe

Token expires: 07/20/2020, 21:52:39
Token expires: 07/20/2020, 21:52:39
Token expires: 07/20/2020, 21:52:39
Token expires: 07/20/2020, 21:52:39
Token expires: 07/20/2020, 21:52:40
Token expires: 07/20/2020, 21:52:40
Token expires: 07/20/2020, 21:52:40


In [10]:
# 3. Get audio data of each of the tracks
track_ids = []
for key, value in dict_of_playlists.items(): #get list of song ids
    track_id_list = value['track_id'].tolist()
    track_ids.append(track_id_list)

In [11]:
track_ids = list(np.concatenate(track_ids).flat) #turn into 1d list

In [12]:
len(track_ids) #can only take 100 at a time

294

In [13]:
# Audio features of interest
audio_feature_track_id = []
acousticness = []
danceability = []
energy = []
instrumentalness = []
key = []
loudness = []
mode = []
tempo = []
valence = []

In [14]:
def get_audio_feature_data(track_id_list):
    audio_feature_data = client.get_resource_list('audio-features', track_id_list)
    for afd in audio_feature_data['audio_features']:
        audio_feature_track_id.append(afd['id'])
        acousticness.append(afd['acousticness'])
        danceability.append(afd['danceability'])
        energy.append(afd['energy'])
        instrumentalness.append(afd['instrumentalness'])
        key.append(afd['key'])
        loudness.append(afd['loudness'])
        mode.append(afd['mode'])
        tempo.append(afd['tempo'])
        valence.append(afd['valence'])

In [15]:
get_audio_feature_data(track_ids[:100])
get_audio_feature_data(track_ids[100:200])
get_audio_feature_data(track_ids[200:])

Token expires: 07/20/2020, 21:52:41
Token expires: 07/20/2020, 21:52:41
Token expires: 07/20/2020, 21:52:42


In [16]:
# Combine lists into dataframe
audio_feature_dict = {
    'audio_feature_track_id': audio_feature_track_id, 'acousticness': acousticness, 'danceability': danceability, 
    'energy': energy, 'instrumentalness': instrumentalness, 'key': key, 'loudness': loudness, 'mode': mode, 
    'tempo': tempo, 'valence': valence
}
df_audio_features = pd.DataFrame(audio_feature_dict)

In [17]:
df_audio_features

,audio_feature_track_id,acousticness,danceability,energy,instrumentalness,key,loudness,mode,tempo,valence
0,1RKZvaLj3UPhGjZkaIrFm7,0.037500,0.795,0.885,0.006380,8,-4.130,1,126.037,0.4600
1,7BlBVFwvbWvcwNcUarQmjk,0.462000,0.780,0.759,0.000868,1,-3.292,1,91.952,0.5790
2,6T0PSfzdg8PE44T8PbbEmM,0.006400,0.775,0.717,0.265000,5,-5.348,0,122.994,0.2820
3,623mI2F1iCO4cqGbv6WUVn,0.112000,0.672,0.733,0.373000,2,-8.457,1,111.436,0.3370
4,3YT3RgSGhvgormc5f7h2xi,0.203000,0.821,0.540,0.000657,3,-7.001,1,114.021,0.5870
...,...,...,...,...,...,...,...,...,...,...
289,5qadNHtHJMEH6Nn061W90f,0.208000,0.787,0.817,0.892000,11,-8.482,0,129.962,0.0376
290,0ueulvkzOC42IJDSNkrxXH,0.000058,0.653,0.873,0.000609,1,-6.299,1,128.056,0.5750
291,4J97CmLk4uoyzW0p6LE4Zl,0.000102,0.474,0.970,0.054200,4,-2.115,0,125.974,0.1790
292,6tNQw4RAj7oYogHXVzKJg8,0.013900,0.468,0.939,0.000092,11,-1.976,1,176.075,0.6980


In [18]:
# Join dataframes on audio feature track id
for key, value in dict_of_playlists.items():
    dict_of_playlists[key] = value.merge(df_audio_features, left_on='track_id', right_on='audio_feature_track_id')

In [19]:
# 4. Get data on the artists of the tracks
artist1_ids = []
artist2_ids = []
artist3_ids = []
for key, value in dict_of_playlists.items(): #get list of song ids
    artist1_list = value['artist1_id'].tolist()
    artist1_ids.append(artist1_list)
    artist2_list = value['artist2_id'].tolist()
    artist2_ids.append(artist2_list)
    artist3_list = value['artist3_id'].tolist()
    artist3_ids.append(artist3_list)

In [20]:
artist1_ids = list(np.concatenate(artist1_ids).flat)
artist2_ids = list(np.concatenate(artist2_ids).flat)
artist3_ids = list(np.concatenate(artist3_ids).flat)

In [21]:
artist2_ids = [x for x in artist2_ids if x is not None] # Remove nones
artist3_ids = [x for x in artist3_ids if x is not None]

In [22]:
print(len(artist1_ids), len(artist2_ids), len(artist3_ids))

294 20 5


In [23]:
# Artist features of interest
artist1_followers = []
artist1_genres = []
artist1_popularity = []
artist2_followers = []
artist2_genres = []
artist2_popularity = []
artist3_followers = []
artist3_genres = []
artist3_popularity = []

In [24]:
def get_artist_data(artist_id_list, artist_num):
    artist_data = client.get_resource_list('artists', artist_id_list)
    for ad in artist_data['artists']:
        if artist_num == 1:
            artist1_followers.append(ad['followers']['total'])
            artist1_genres.append(ad['genres'])
            artist1_popularity.append(ad['popularity'])
        elif artist_num == 2:
            artist2_followers.append(ad['followers']['total'])
            artist2_genres.append(ad['genres'])
            artist2_popularity.append(ad['popularity'])
        elif artist_num == 3:
            artist3_followers.append(ad['followers']['total'])
            artist3_genres.append(ad['genres'])
            artist3_popularity.append(ad['popularity'])

In [25]:
get_artist_data(artist1_ids[:50], 1)
get_artist_data(artist1_ids[50:100], 1)
get_artist_data(artist1_ids[100:150], 1)
get_artist_data(artist1_ids[150:200], 1)
get_artist_data(artist1_ids[200:250], 1)
get_artist_data(artist1_ids[250:], 1)
get_artist_data(artist2_ids, 2)
get_artist_data(artist3_ids, 3)

Token expires: 07/20/2020, 21:52:42
Token expires: 07/20/2020, 21:52:43
Token expires: 07/20/2020, 21:52:43
Token expires: 07/20/2020, 21:52:43
Token expires: 07/20/2020, 21:52:43
Token expires: 07/20/2020, 21:52:44
Token expires: 07/20/2020, 21:52:44
Token expires: 07/20/2020, 21:52:44


In [26]:
# Combine lists into dataframe
artist1_dict = {
    'track_id': audio_feature_track_id,'artist1_id': artist1_ids, 'artist1_followers': artist1_followers, 
    'artist1_genres': artist1_genres, 'artist1_popularity': artist1_popularity 
}
df_artist1 = pd.DataFrame(artist1_dict)

artist2_dict = {
    'artist2_id': artist2_ids, 'artist2_followers': artist2_followers, 
    'artist2_genres': artist2_genres, 'artist2_popularity': artist2_popularity 
}
df_artist2 = pd.DataFrame(artist2_dict)

artist3_dict = {
    'artist3_id': artist3_ids, 'artist3_followers': artist3_followers, 
    'artist3_genres': artist3_genres, 'artist3_popularity': artist3_popularity 
}
df_artist3 = pd.DataFrame(artist3_dict)

In [27]:
df_artist1

,track_id,artist1_id,artist1_followers,artist1_genres,artist1_popularity
0,1RKZvaLj3UPhGjZkaIrFm7,738wLrAtLtCtFOLvQBXOXp,6028900,"[dance pop, edm, electro house, moombahton, po...",82
1,7BlBVFwvbWvcwNcUarQmjk,7ltDVBr6mKbRvohxheJ9h1,3699511,[r&b en espanol],84
2,6T0PSfzdg8PE44T8PbbEmM,586uxXMyD5ObPuzjtrzO1Q,405831,"[electropop, pop]",73
3,623mI2F1iCO4cqGbv6WUVn,0MoaBi6dSquXp6rrlqlF8R,40030,"[indietronica, latintronica, puerto rican indie]",49
4,3YT3RgSGhvgormc5f7h2xi,1hzfo8twXdOegF3xireCYs,1348476,[german pop],76
...,...,...,...,...,...
289,5qadNHtHJMEH6Nn061W90f,6ODCVWBfGNFUf1bpo0c2Ge,480668,"[afrofuturismo brasileiro, baile pop, brazilia...",55
290,0ueulvkzOC42IJDSNkrxXH,1puzPcrOHKTMQ6KTuYfqU0,4616,[funk carioca],44
291,4J97CmLk4uoyzW0p6LE4Zl,7ybnRZKtRbWQzJfKyYSyh6,28570,"[alternative americana, modern alternative rock]",49
292,6tNQw4RAj7oYogHXVzKJg8,3jOstUTkEu2JkjvRdBA5Gu,2040178,"[alternative rock, modern power pop, modern ro...",76


In [28]:
len(set(artist3_ids))

5

In [29]:
for key, value in dict_of_playlists.items():
    dict_of_playlists[key] = value.merge(df_artist1, left_on='track_id', right_on='track_id')

for key, value in dict_of_playlists.items():
    dict_of_playlists[key] = value.merge(df_artist2, how='left', left_on='artist2_id', right_on='artist2_id')

for key, value in dict_of_playlists.items():
    dict_of_playlists[key] = value.merge(df_artist3, how='left', left_on='artist3_id', right_on='artist3_id')

In [30]:
# 5. Get data on the albums the tracks are in
album_ids = []
for key, value in dict_of_playlists.items():
    album_id_list = value['album_id'].tolist()
    album_ids.append(album_id_list)

In [31]:
album_ids = list(np.concatenate(album_ids).flat)

In [32]:
print(len(album_ids), len(set(album_ids)))

294 293


In [33]:
# Album features of interest
album_type = []
album_popularity = []
album_release_date = []

In [34]:
def get_album_data(album_id_list):
    album_data = client.get_resource_list('albums', album_id_list)
    for ad in album_data['albums']:
        album_type.append(ad['album_type'])
        album_popularity.append(ad['popularity'])
        album_release_date.append(ad['release_date'])

In [35]:
for i in range(15):
    get_album_data(album_ids[i*20:i*20+20])

Token expires: 07/20/2020, 21:52:45
Token expires: 07/20/2020, 21:52:45
Token expires: 07/20/2020, 21:52:45
Token expires: 07/20/2020, 21:52:46
Token expires: 07/20/2020, 21:52:46
Token expires: 07/20/2020, 21:52:46
Token expires: 07/20/2020, 21:52:47
Token expires: 07/20/2020, 21:52:47
Token expires: 07/20/2020, 21:52:47
Token expires: 07/20/2020, 21:52:48
Token expires: 07/20/2020, 21:52:48
Token expires: 07/20/2020, 21:52:48
Token expires: 07/20/2020, 21:52:48
Token expires: 07/20/2020, 21:52:49
Token expires: 07/20/2020, 21:52:49


In [36]:
# Combine all dataframes into one large dataframe
album_dict = {
    'track_id': audio_feature_track_id, 'album_type': album_type, 
    'album_popularity': album_popularity, 'album_release_date': album_release_date
}
df_album = pd.DataFrame(album_dict)

In [37]:
df_album

,track_id,album_type,album_popularity,album_release_date
0,1RKZvaLj3UPhGjZkaIrFm7,single,66,2019-09-11
1,7BlBVFwvbWvcwNcUarQmjk,single,73,2019-08-15
2,6T0PSfzdg8PE44T8PbbEmM,single,13,2019-07-12
3,623mI2F1iCO4cqGbv6WUVn,single,39,2019-09-25
4,3YT3RgSGhvgormc5f7h2xi,single,36,2019-09-27
...,...,...,...,...
289,5qadNHtHJMEH6Nn061W90f,album,26,2013
290,0ueulvkzOC42IJDSNkrxXH,single,0,2015-07-22
291,4J97CmLk4uoyzW0p6LE4Zl,album,46,2017-08-04
292,6tNQw4RAj7oYogHXVzKJg8,single,39,2017-03-16


In [38]:
for key, value in dict_of_playlists.items():
    dict_of_playlists[key] = value.merge(df_album, left_on='track_id', right_on='track_id')

In [39]:
dict_of_playlists['FIFA 20 Soundtrack']

,playlist_name,num_tracks,track_name,track_duration,track_explicit,track_popularity,track_id,album_name,album_endpoint,album_id,...,artist1_popularity,artist2_followers,artist2_genres,artist2_popularity,artist3_followers,artist3_genres,artist3_popularity,album_type,album_popularity,album_release_date
0,FIFA 20 Soundtrack,41,Que Calor (feat. J Balvin & El Alfa),169997,False,73,1RKZvaLj3UPhGjZkaIrFm7,Que Calor (feat. J Balvin & El Alfa),https://api.spotify.com/v1/albums/5AKSUA3pizho...,5AKSUA3pizhoVgfjW1ijVC,...,82,21030091.0,"[latin, reggaeton]",95.0,1030909.0,"[dembow, latin, rap dominicano, reggaeton, tro...",83.0,single,66,2019-09-11
1,FIFA 20 Soundtrack,41,"Yo x Ti, Tu x Mi",201040,False,80,7BlBVFwvbWvcwNcUarQmjk,"Yo x Ti, Tu x Mi",https://api.spotify.com/v1/albums/3844bY26oeSk...,3844bY26oeSkqd06th4EYp,...,84,23525333.0,"[latin, puerto rican pop, reggaeton, trap latino]",93.0,NaN,NaN,NaN,single,73,2019-08-15
2,FIFA 20 Soundtrack,41,Swing,179658,False,19,6T0PSfzdg8PE44T8PbbEmM,Swing,https://api.spotify.com/v1/albums/23BlliKjsR6e...,23BlliKjsR6eGFuBit4R7K,...,73,NaN,NaN,NaN,NaN,NaN,NaN,single,13,2019-07-12
3,FIFA 20 Soundtrack,41,Vámono,208329,False,46,623mI2F1iCO4cqGbv6WUVn,Vámono,https://api.spotify.com/v1/albums/0DAtzeZ09MxU...,0DAtzeZ09MxUORBOwGWxF5,...,49,NaN,NaN,NaN,NaN,NaN,NaN,single,39,2019-09-25
4,FIFA 20 Soundtrack,41,Fado,250400,False,42,3YT3RgSGhvgormc5f7h2xi,Fado,https://api.spotify.com/v1/albums/1ZZwX6IB8WGD...,1ZZwX6IB8WGDPB314R8yXl,...,76,NaN,NaN,NaN,NaN,NaN,NaN,single,36,2019-09-27
5,FIFA 20 Soundtrack,41,Offence,168521,True,54,4cakdyTV2GRb9ktqBNMChH,GREY Area,https://api.spotify.com/v1/albums/4Wwm4xg2748z...,4Wwm4xg2748zhYuzDRFTgY,...,62,NaN,NaN,NaN,NaN,NaN,NaN,album,61,2019-03-01
6,FIFA 20 Soundtrack,41,Unemployed,168561,False,36,4Qz6dwfC5VRaOPBw9ZbWzM,Unemployed,https://api.spotify.com/v1/albums/4wT85WYKlyU9...,4wT85WYKlyU9XZddZAffSq,...,70,NaN,NaN,NaN,NaN,NaN,NaN,single,30,2019-03-19
7,FIFA 20 Soundtrack,41,RUNAWAY,161759,False,28,6iRXOuVt9bOdGr3nohzddj,RUNAWAY,https://api.spotify.com/v1/albums/3bdA73eL6cN5...,3bdA73eL6cN51aC2rPFfyc,...,65,NaN,NaN,NaN,NaN,NaN,NaN,single,23,2019-06-13
8,FIFA 20 Soundtrack,41,Ozone,197248,True,47,6mOHAShVtqwJIBRAd7Xa4l,Ozone,https://api.spotify.com/v1/albums/4x9Nh0dyy93t...,4x9Nh0dyy93tsBOQylu5pM,...,40,NaN,NaN,NaN,NaN,NaN,NaN,single,40,2019-05-03
9,FIFA 20 Soundtrack,41,Frens,198849,False,41,15FfKpV7MURQncDusPvqwM,Frens,https://api.spotify.com/v1/albums/6ptrrZitJ1n2...,6ptrrZitJ1n2a6qraQGmLz,...,51,NaN,NaN,NaN,NaN,NaN,NaN,single,34,2019-03-14


In [40]:
# Drop unnecessary columns and combine dataframes
df = pd.DataFrame()
for key, value in dict_of_playlists.items():
    df = pd.concat([df,value], ignore_index=True)

In [41]:
df.columns

Index(['playlist_name', 'num_tracks', 'track_name', 'track_duration',
       'track_explicit', 'track_popularity', 'track_id', 'album_name',
       'album_endpoint', 'album_id', 'artist1_name', 'artist1_endpoint',
       'artist1_id_x', 'artist2_name', 'artist2_endpoint', 'artist2_id',
       'artist3_name', 'artist3_endpoint', 'artist3_id',
       'audio_feature_track_id', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'loudness', 'mode', 'tempo', 'valence',
       'artist1_id_y', 'artist1_followers', 'artist1_genres',
       'artist1_popularity', 'artist2_followers', 'artist2_genres',
       'artist2_popularity', 'artist3_followers', 'artist3_genres',
       'artist3_popularity', 'album_type', 'album_popularity',
       'album_release_date'],
      dtype='object')

In [42]:
df = df.drop(['track_id','album_endpoint','album_id','artist1_endpoint','artist1_id_x','artist2_endpoint',
         'artist2_id','artist3_endpoint','artist3_id','audio_feature_track_id','artist1_id_y'], axis=1)

In [43]:
cols = ['playlist_name','num_tracks','track_name','track_duration','track_explicit','track_popularity',
        'acousticness','danceability','energy','instrumentalness','key','loudness','mode','tempo','valence',
        'artist1_name','artist1_followers','artist1_genres','artist1_popularity',
        'artist2_name','artist2_followers','artist2_genres','artist2_popularity',
        'artist3_name','artist3_followers','artist3_genres','artist3_popularity',
        'album_name','album_type','album_popularity','album_release_date']

In [44]:
df = df[cols] #reorder columns

In [45]:
df

,playlist_name,num_tracks,track_name,track_duration,track_explicit,track_popularity,acousticness,danceability,energy,instrumentalness,...,artist2_genres,artist2_popularity,artist3_name,artist3_followers,artist3_genres,artist3_popularity,album_name,album_type,album_popularity,album_release_date
0,FIFA 20 Soundtrack,41,Que Calor (feat. J Balvin & El Alfa),169997,False,73,0.037500,0.795,0.885,0.006380,...,"[latin, reggaeton]",95.0,El Alfa,1030909.0,"[dembow, latin, rap dominicano, reggaeton, tro...",83.0,Que Calor (feat. J Balvin & El Alfa),single,66,2019-09-11
1,FIFA 20 Soundtrack,41,"Yo x Ti, Tu x Mi",201040,False,80,0.462000,0.780,0.759,0.000868,...,"[latin, puerto rican pop, reggaeton, trap latino]",93.0,None,NaN,NaN,NaN,"Yo x Ti, Tu x Mi",single,73,2019-08-15
2,FIFA 20 Soundtrack,41,Swing,179658,False,19,0.006400,0.775,0.717,0.265000,...,NaN,NaN,None,NaN,NaN,NaN,Swing,single,13,2019-07-12
3,FIFA 20 Soundtrack,41,Vámono,208329,False,46,0.112000,0.672,0.733,0.373000,...,NaN,NaN,None,NaN,NaN,NaN,Vámono,single,39,2019-09-25
4,FIFA 20 Soundtrack,41,Fado,250400,False,42,0.203000,0.821,0.540,0.000657,...,NaN,NaN,None,NaN,NaN,NaN,Fado,single,36,2019-09-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,FIFA 18 Soundtrack,45,Caxambu,175000,False,22,0.208000,0.787,0.817,0.892000,...,NaN,NaN,None,NaN,NaN,NaN,Batuk Freak (Instrumentals),album,26,2013
290,FIFA 18 Soundtrack,45,Deela Ding,196937,False,0,0.000058,0.653,0.873,0.000609,...,NaN,NaN,None,NaN,NaN,NaN,Vira a Cara,single,0,2015-07-22
291,FIFA 18 Soundtrack,45,11 Acre,197413,False,32,0.000102,0.474,0.970,0.054200,...,NaN,NaN,None,NaN,NaN,NaN,Plastic Soul,album,46,2017-08-04
292,FIFA 18 Soundtrack,45,Feels Like Summer,195389,False,46,0.013900,0.468,0.939,0.000092,...,NaN,NaN,None,NaN,NaN,NaN,Feels Like Summer,single,39,2017-03-16


In [46]:
df.to_csv('fifa_spotify_data')